## Collaborative Filtering
- Reference: https://www.kaggle.com/code/julian3833/h-m-collaborative-filtering-user-user

In [ ]:
import time
import numpy as np
import pandas as pd

import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_path = '/content/drive/My Drive/AIPI 590/Final Project/data/'
training = f'{base_path}transactions_train.csv'
submission = f'{base_path}sample_submission.csv'
customers = f'{base_path}customers.csv'
articles = f'{base_path}articles.csv'

df = pd.read_csv(training, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(submission)

In [ ]:
users = pd.read_csv(customers)
items = pd.read_csv(articles, dtype={'article_id': str})

unique_users = users['customer_id'].unique().tolist()
unique_items = items['article_id'].unique().tolist()

user_to_customer_map = {user_id: customer_id for user_id, customer_id in enumerate(unique_users)}
customer_to_user_map = {customer_id: user_id for user_id, customer_id in enumerate(unique_users)}

item_to_article_map = {item_id: article_id for item_id, article_id in enumerate(unique_items)}
article_to_item_map = {article_id: item_id for item_id, article_id in enumerate(unique_items)}

In [ ]:
df['user_id'] = df['customer_id'].map(customer_to_user_map)
df['item_id'] = df['article_id'].map(article_to_item_map)

In [ ]:
from random import sample
sampled_id = sample([i for i in range(df.shape[0])], k = int(df.shape[0]/2))

In [ ]:
df_sample = df.iloc[sampled_id, :]

In [ ]:
# Configuration Parameters
N_SIMILAR_USERS = 30
MINIMUM_PURCHASES = 3
START_DATE = '2020-08-21'
DROP_PURCHASED_ITEMS = False
DROP_USER_FROM_HIS_NEIGHBORHOOD = False
TEST_RUN = False
TEST_SIZE = 1000

In [ ]:
def flatten(l):
    """ Flatten a list of lists"""
    return [item for sublist in l for item in sublist]

def compare_vectors(v1, v2):
    """Compare lists of purchased product for two given users
    v1 stands for the "vector representation for user 1", which is a list of the purchases of u1
    
    Returns:
        A value between 0 and 1 (similarity)
    """
    intersection = len(set(v1) & set(v2))
    denominator = np.sqrt(len(v1) * len(v2))
    return intersection / denominator

def get_similar_users(u, v, dfh):
    """
    Get the N_SIMILAR_USERS most similar users to the given one with their similarity score
    Arguments:
        u: the user_id, 
        v:  the "vector" representation of the user (list of item_id)
        dfh : the "history of transaccions" dataframe
        
    Returns:
        tuple of lists ([similar user_id], [similarity scores])
    """
    similar_users = dfh.apply(lambda v_other: compare_vectors(v, v_other)).sort_values(ascending=False).head(N_SIMILAR_USERS + 1)
    
    if DROP_USER_FROM_HIS_NEIGHBORHOOD:
        similar_users = similar_users[similar_users.index != u]
        
    return similar_users.index.tolist(), similar_users.tolist()

def get_items(u, v, dfh):
    """ Get the recommend items for a given users
    
    It will:
        1) Get similar users for the given user
        2) Obtain all the items those users purchased
        3) Rank them using the similarity scores of the user that purchased them
        4) Return the 12 best ranked
    
    Arguments:
        u: the user_id, 
        v:  the "vector" representation of the user (list of item_id)
        dfh : the "history of transaccions" dataframe
        
    Returns:
        list of item_id of lenght at most 12
    """
    global i, n
    
    users, scores = get_similar_users(u, v, dfh)
    df_nn = pd.DataFrame({'user': users, 'score': scores})
    df_nn['items'] = df_nn.apply(lambda row: dfh.loc[row.user], axis=1)
    df_nn['weighted_items'] = df_nn.apply(lambda row: [(item, row.score) for item in row['items']], axis=1)

    recs = pd.DataFrame(flatten(df_nn['weighted_items'].tolist()), columns=['item', 'score']).groupby('item')['score'].sum().sort_values(ascending=False)
    if DROP_PURCHASED_ITEMS:
        recs = recs[~recs.index.isin(v)]
    # Keep the first 12 and get the item_ids
    i +=1
    if i % 200 == 0:
        pid = mp.current_process().pid
        print(f"[PID {pid:>2d}] Finished {i:3d} / {n:5d} - {i/n*100:3.0f}%")
    return recs.head(12).index.tolist()

def get_items_chunk(user_ids: np.array, dfh: pd.DataFrame):
    """ Call get_item for a list of user_ids
    
    Arguments:
        user_ids: list of user_id, 
        dfh: the "history of transaccions" dataframe
        
    Returns:
        pd.Series with index user_id and list of item_id (recommendations) as value
    """
    global i, n
    i = 0
    
    n = len(user_ids)
    pid = mp.current_process().pid
    print(f"[PID {pid:>2d}] Started working with {n:5d} users")
    
    df_user_vectors = pd.DataFrame(dfh.loc[user_ids]).reset_index()
    df_user_vectors['recs'] = df_user_vectors.apply(lambda row: get_items(row.user_id, row.item_id, dfh), axis=1)
    return df_user_vectors.set_index('user_id')['recs']

def get_recommendations(users: list, dfh: pd.DataFrame):
    """
    Obtained recommendation for the users using transaccion dfh in a parallelized manner
    
    Call get_items_chunk in a "smart" multiprocessing fashion
    
    Arguments:
        users: list of user_id
        dfh: the "history of transaccions" dataframe
    
    Returns:
        pd.DataFrame with index user_id and list of item_id (recommendations) as value
    
    """
    time_start = time.time()
    
    # Split into approximately evenly sized chunks
    # We will send just one batch to each CPU 
    user_chunks = np.array_split(users, mp.cpu_count())
    
    f = partial(get_items_chunk, dfh=dfh)
    with Pool(mp.cpu_count()) as p:
        res = p.map(f, user_chunks)
    
    df_rec = pd.DataFrame(pd.concat(res))

    elapsed = (time.time() - time_start) / 60
    print(f"Finished get_recommendations({len(users)}). It took {elapsed:5.2f} mins")
    return df_rec


def uucf(df, start_date=START_DATE):
    """ Entry point for the UUCF model. 
    
    Receive the original transactions_train.csv and a start_date and gets UUCF recommendations
    
    The model will not cover the full list of users, but just a subset of them.
    
    It will provide recommendations for users with at least MINIMUM_PURCHASES after start_date.
    It might return less than 12 recs per user.
    
    An ad-hoc function for filling these gaps should be used downstream.
    (See fill functionality right below)
    
    
    Arguments:
        df: The raw dataframe from transactions_train.csv
        start_date: a date
        
    Returns:
        a submission-like pd.DataFrame with columns [customer_id, prediction]
        'prediction' is a list and not a string though
    
    """
    df_filtered = df[df['t_dat'] > start_date]
    print(f"Kept data from {start_date} on. Total rows: {len(df_filtered)}")
    
    # H stands for "Transaction history"
    # Create a dataframe that contains a list of items purchased by the user (user transaction history)
    df_trans = df_filtered.groupby("user_id")['item_id'].apply(lambda items: list(set(items)))
    # Filtered out users that do not meet the minimum purchase requirement
    df_trans = df_trans[df_trans.str.len() >= MINIMUM_PURCHASES]
    if TEST_RUN:
        print("WARNING: TEST_RUN is True. It will be a toy execution.")
        df_trans = df_trans.head(TEST_SIZE)
    
    users = df_trans.index.tolist()
    n_users = len(users)
    print(f"Total  in the time frame with at least {MINIMUM_PURCHASES}: {n_users}")
    
    df_rec = get_recommendations(users, df_trans)
    df_rec['customer_id'] = df_rec.index.map(user_to_customer_map)
    df_rec['prediction'] = df_rec['recs'].map(lambda l: [item_to_article_map[i] for i in l])
    
    # Submission ready dataframe
    df_rec.reset_index(drop=True)[['customer_id', 'prediction']]
    return df_rec 

In [ ]:
df_recs = uucf(df_sample)

Kept data from 2020-08-21 on. Total rows: 595381
Total  in the time frame with at least 3: 75732
[PID 348] Started working with 37866 users
[PID 347] Started working with 37866 users
[PID 347] Finished 200 / 37866 -   1%
[PID 348] Finished 200 / 37866 -   1%
[PID 347] Finished 400 / 37866 -   1%
[PID 348] Finished 400 / 37866 -   1%
[PID 348] Finished 600 / 37866 -   2%
[PID 347] Finished 600 / 37866 -   2%
[PID 348] Finished 800 / 37866 -   2%
[PID 347] Finished 800 / 37866 -   2%
[PID 348] Finished 1000 / 37866 -   3%
[PID 347] Finished 1000 / 37866 -   3%
[PID 348] Finished 1200 / 37866 -   3%
[PID 347] Finished 1200 / 37866 -   3%
[PID 348] Finished 1400 / 37866 -   4%
[PID 347] Finished 1400 / 37866 -   4%
[PID 348] Finished 1600 / 37866 -   4%
[PID 347] Finished 1600 / 37866 -   4%
[PID 348] Finished 1800 / 37866 -   5%
[PID 347] Finished 1800 / 37866 -   5%
[PID 348] Finished 2000 / 37866 -   5%
[PID 347] Finished 2000 / 37866 -   5%
[PID 348] Finished 2200 / 37866 -   6%
[PID 3

In [ ]:
bench_mark = f'{base_path}benchmark.csv'
df_fill = pd.read_csv(bench_mark)

In [ ]:
def drop_duplicates(seq):
    """ Remove duplicates of a given sequence keeping order"""
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def fill_row(row):
    uucf = row['prediction_uucf']
    fill = row['prediction_fill'].split()
    new_list = drop_duplicates(uucf + fill)[:12]
    return ' '.join(new_list)


def fill(df_recs, df_fill):
    df_recs['len'] = df_recs['prediction'].str.len()
    df_recs = pd.merge(df_fill, df_recs, how='left', on='customer_id', suffixes=('_fill', '_uucf'))
    
    
    # No recs from UUCF at all: use the fallback model 
    df_recs.loc[df_recs['prediction_uucf'].isnull(), 'prediction'] = df_recs['prediction_fill']


    # Full UUCF recommendation
    mask = df_recs['prediction_uucf'].notnull() & (df_recs['len'] == 12)
    df_recs.loc[mask, 'prediction'] = df_recs['prediction_uucf']


    # Fill with another model. Not enough recs from UUCF
    fill_mask = df_recs['prediction_uucf'].notnull() & (df_recs['len'] < 12)
    df_recs.loc[fill_mask, 'prediction'] = df_recs[fill_mask].apply(fill_row, axis=1)
    return df_recs.drop(['prediction_uucf', 'prediction_fill', 'len', 'recs'], axis=1)

In [ ]:
# Fill with another model
df_sub = fill(df_recs, df_fill)
df_sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0909370001 0915526001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0923758001 0918522001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0923758001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0923758001 0918522001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0923758001 0918522001 09...


In [ ]:
# Submit
df_sub.to_csv("uucf.csv", index=False)
!cp uucf.csv "drive/My Drive/"